In [2]:
from http.server import BaseHTTPRequestHandler, HTTPServer
import logging
import requests
import urllib.request
import json
import numpy as np
from tensorflow.keras.models import load_model
from gensim.models import FastText
from konlpy.tag import Okt
import boto3
import sys
import pandas as pd
from tqdm import tqdm
from keras.preprocessing.text import Tokenizer
from keras.preprocessing import sequence
import urllib.parse
import urllib.request
from bs4 import BeautifulSoup
from sklearn.feature_extraction.text import TfidfVectorizer

MAX_NB_WORDS = 100000
SNU_model = load_model('SNU_LSTM_Model.h5')
# SNU_model_shuffle = load_model('SNU_LSTM_Model_shuffle.h5')
similar_model = FastText.load('similar_keyword_model')

class MyTokenizer:
    def __init__(self, tagger):
        self.tagger = tagger

    def __call__(self, sent):
        pos = self.tagger.pos(sent)
        clean_words = []  # 정제된 단어 리스트
        for word in pos:
            # word[1]은 품사를 의미하며, 여기서는 조사, 문장기호, 접두사, Foreign('\n'을 빼주기 위함)인 것은 제외시킴.
            if word[1] not in ['Josa', 'Punctuation', 'Suffix', 'Foreign']:
                if len(word[0]) >= 2:  # 한 글자인 단어들도 의미가 없는 경우가 많으므로 일단 제외.
                    #if word[0] not in ['있다', '했다', '한다', '없다', '된다']:
                    clean_words.append(word[0])
        return clean_words


my_Tokenizer = MyTokenizer(Okt())


In [3]:
import pymysql
import sys
import os
import requests
import base64
import json
import logging
import time
import pandas as pd
import csv
import datetime

host = '' # rds endpoint
port = 3306
username = ''
database = ''
password = ''

def connect_RDS(host, port, username, password, database):
    try:
        conn = pymysql.connect(host = host, user = username, password = password, db = database, port = port, use_unicode = True, charset = 'utf8')
        cursor = conn.cursor()
    
    except:
        logging.error('RDS에 연결되지 않았습니다.')
        sys.exit(1)
        
    return conn, cursor

In [4]:
if sys.version_info[0] < 3:
    from io import StringIO # Python 2.x
else:
    from io import StringIO # Python 3.x

# get your credentials from environment variables
aws_id = ''
aws_secret = ''

client = boto3.client('s3', aws_access_key_id=aws_id,
        aws_secret_access_key=aws_secret)

bucket_name = 'snucsv'

object_key = 'SNU_Data_1200.csv'
csv_obj = client.get_object(Bucket=bucket_name, Key=object_key)
body = csv_obj['Body']
csv_string = body.read().decode('cp949')

# load data
train_df = pd.read_csv(StringIO(csv_string))
label_names = ["label"]
y_train = train_df[label_names].values

train_df['doc_len'] = train_df['document'].apply(lambda words: len(words.split(" ")))
max_seq_len = np.round(train_df['doc_len'].mean() + train_df['doc_len'].std()).astype(int)
train_df

,document,label,doc_len
0,현재 AI기술은 불완전하기 때문에 가짜뉴스 완전히 걸러낼 수 없다,1,9
1,"‘정치 세대교체’ 바람 일으킨 이준석, 대선엔 출마 못한다",1,8
2,액상형 전자담배는 니코틴이 함유되지 않아 담배가 아니다,1,7
3,한국 백신 접종률 세계 100위권 이하다,1,6
4,광역도시 없는 지역권은 실제 수요와 관계없이 획일적으로 국가교통망 정책에서 소외되고 있다,1,11
...,...,...,...
1219,""" ‘당·정·청 전원회의’는 '운동권 용어'다""?",0,5
1220,"""늘어가는 해양사고, 전북 관내 잠수구조인력 전무(全無)""?",0,6
1221,"""청와대 하루 평균 업무추진비, 이명박 768만원, 박근혜 814만원, 문재인 55만원""",0,10
1222,"국민연금공단 기금운용본부가 전주로 이전한 후, 해외투자자가 국민연금을 '패싱'한다",0,8


**단어 벡터화 및 패딩**

In [5]:
raw_docs_train = train_df['document'].tolist()
num_classes = len(label_names)
print(num_classes)
processed_docs_train = []

for doc in tqdm(raw_docs_train):
    tokens = my_Tokenizer(doc)
    processed_docs_train.append(tokens)

tokenizer = Tokenizer(num_words=MAX_NB_WORDS, lower=True, char_level=False)
tokenizer.fit_on_texts(processed_docs_train)
word_seq_train = tokenizer.texts_to_sequences(processed_docs_train)
word_index = tokenizer.word_index
print('dictionary size :', len(word_index))

word_seq_train = sequence.pad_sequences(word_seq_train, maxlen=max_seq_len)
word_seq_train

  0%|          | 0/1224 [00:00<?, ?it/s]

1


100%|██████████| 1224/1224 [00:09<00:00, 127.99it/s]

dictionary size : 3999


array([[ 503, 1459,  504, ..., 1461, 1462,    8],
       [   0,   72,  761, ...,   51,  360,    5],
       [   0,    0, 1466, ...,  506,  763,   41],
       ...,
       [ 649,  177,  381, ..., 3992,   13, 3993],
       [3994, 3995, 3996, ...,  248, 3998,    5],
       [   0,    0,    0, ..., 1401, 3999,  496]], dtype=int32)

**사용자에게 받은 데이터 전처리 테스트**

In [7]:
response = ['현재 AI기술은 불완전하기 때문에 가짜뉴스 완전히 걸러낼 수 없다', 'ajsdlkfakljflalejlka', 'dshkjlafjsdkfjalks']
tokens_response = []

for i in range(len(response)):
    tokens = my_Tokenizer(response[i])
    tokens_response.append(tokens)
#     print(tokens_response)

word_seq_response = tokenizer.texts_to_sequences(tokens_response)
print(word_seq_response[0])
word_seq_response = sequence.pad_sequences(word_seq_response, maxlen=max_seq_len)
print(word_seq_response.shape)

score_list = []
for sco in range(len(word_seq_response)):
    word_seq_response_to_score = word_seq_response[sco].reshape(1,max_seq_len)
    print(word_seq_response_to_score)
    score_list.append(float(SNU_model.predict(word_seq_response_to_score)))
print(score_list)

[503, 1459, 504, 1460, 22, 71, 104, 1461, 1462, 8]
(3, 10)
[[ 503 1459  504 1460   22   71  104 1461 1462    8]]
[[0 0 0 0 0 0 0 0 0 0]]
[[0 0 0 0 0 0 0 0 0 0]]
[0.5054650902748108, 0.5058199763298035, 0.5058199763298035]


**입력 데이터 처리 함수**

In [11]:
# ===========================================================================================================================
# 단어 토큰화 및 빈도순 정렬 후 추출

def tfidf_Vectorizer(response_list):
    tfidf_Vectorizer = TfidfVectorizer(tokenizer=my_Tokenizer, min_df=1) # df 값(단어가 몇 문장들에서 등장하였는지)을 최소 'min_df' 값으로 설정.
    X = tfidf_Vectorizer.fit_transform(response_list).toarray()
    print(X.shape)    # X(2차원 배열)의 행,열 수를 출력.
    print(tfidf_Vectorizer.vocabulary_)   # 각 단어들이 배열 X에서 몇번째 열(인덱스 값)에 해당하는지 출력.


    #pandas를 활용하여 각 단어들의 각 문장에서의 tf-idf 값들을 모두 더하고, 내림차순으로 정렬하여 상위 n개 출력
    count = X.sum(axis=0)    # 2차원 배열 X에서 각 열을 기준으로 합을 구함. (각 단어들의 '최종' tf-idf 값으로 간주.)
    word_count = pd.DataFrame({
        '단어' : tfidf_Vectorizer.get_feature_names(),
        '빈도' : count.flat
    })
    sorted_df = word_count.sort_values('빈도', ascending=False)
    print(sorted_df.head(10), "\n")

    word_ = list(np.array(sorted_df['단어'].tolist()))
    return(word_)


# ===========================================================================================================================
# 연관키워드 추출

def similar_keyword(word_):
    similar_list = []

    if len(word_) == 1:
        similar_keywords = similar_model.wv.most_similar(word_[0])

        for i in range(len(similar_keywords)):
            similar_list.append(similar_keywords[i][0])
    return similar_list


# ===========================================================================================================================
# MySQL 데이터베이스에 응답데이터, 시간 저장

def MySQL_data_save(response):
    global cursor, conn
    conn, cursor = connect_RDS(host, port, username, password, database)

    now = datetime.datetime.now()
    Korea_DatetimeNow = now + datetime.timedelta(hours = 9)
    DatetimeNow = Korea_DatetimeNow.strftime('%Y-%m-%d %H:%M:%S')

    try:
        with conn.cursor() as cursor:
            query = "INSERT INTO factcheck.article_data (article_data, date) VALUES(%s, %s)"
            val = (response, DatetimeNow)
            cursor.execute(query, val)
            conn.commit()

    finally:
        conn.close()
        
        
# ===========================================================================================================================
# MySQL 데이터베이스에 키워드 랭킹 반영

def keyword_rank(word_):
    rank_list = []
    conn, cursor = connect_RDS(host, port, username, password, database)

    try:
        with conn.cursor() as cursor:
            cursor.execute('SELECT keyword FROM keyword_rank')
            row = cursor.fetchall()

        for i in range(len(row)):
            rank_list.append(row[i][0])

        with conn.cursor() as cursor:
            for i in range(0,1):
                if word_[i] in rank_list:
                    query = "UPDATE keyword_rank SET count = count + 1 WHERE keyword = (%s)"
                    val = (word_[i])
                    cursor.execute(query, val)
                    conn.commit()
                else:
                    query = "INSERT INTO factcheck.keyword_rank (keyword, count) VALUES(%s, 1)"
                    val = (word_[i])
                    cursor.execute(query, val)
                    conn.commit()

        top10_list = []
        with conn.cursor() as cursor:
            cursor.execute("SELECT keyword FROM keyword_rank ORDER BY count DESC LIMIT 10")
            row_top = cursor.fetchall()
            for i in range(len(row_top)):
                top10_list.append(row_top[i][0])

    finally:
        conn.close()
    return top10_list


# ===========================================================================================================================
# SNU 기사 검색      

def SNU_Search(word_):
    title_list2 = []
    # link_list2 = []
    score_list2 = []

    try:
#             str_expr = "document.str.contains('{}', case=False)".format(word_[0]) and "document.str.contains('{}', case=False)".format(word_[1])
        str_expr = "document.str.contains('{}', case=False)".format(word_[0])

        df_q = train_df.query(str_expr, engine="python")

        title = list(np.array(df_q['document'].tolist()))
        score_SNU = list(np.array(df_q['label'].tolist()))
        # link = list(np.array(df_q['link'].tolist()))

        for i in range(len(title)):
            title_list2.append(title[i])
        #     link_list2.append(link[i])
            score_list2.append(score_SNU[i])

    except:
        pass
    return title_list2, score_list2


# ===========================================================================================================================
# 네이버 뉴스 검색

def Naver_Search(response):
    baseurl = "https://search.naver.com/search.naver?where=nexearch&sm=top_hty&fbm=1&ie=utf8&query="

    url = baseurl + urllib.parse.quote_plus(response)

    html = urllib.request.urlopen(url).read()
    soup = BeautifulSoup(html, 'html.parser')

    title = soup.find_all(class_ = 'news_tit')

    title_list = []
    link_list = []

    for i in title:
        title_list.append(i.attrs['title'])
        link_list.append(i.attrs['href'])

# ================================================================

    tokens_response = []
    tokens_response2 = []
    score_list = []

    for i in range(len(title_list)):
        tokens = my_Tokenizer(title_list[i])
        tokens_response.append(tokens)

    word_seq_response = tokenizer.texts_to_sequences(tokens_response)
    word_seq_response = sequence.pad_sequences(word_seq_response, maxlen=max_seq_len)

    for sco in range(len(title_list)):
        word_seq_response_to_score = word_seq_response[sco].reshape(1,max_seq_len)
        score_list.append(float(SNU_model.predict(word_seq_response_to_score)))
    
    return title_list, link_list, score_list

**Flask 웹 서버**

In [14]:
#-*-coding:utf-8-*-

from flask import Flask, request
import json
from urllib import parse

app = Flask(__name__)
@app.route('/', methods = ['POST'])

def hello():
    if request.method == 'POST':
        params = request.get_data()
        params_decode = str(params.decode('utf-8'))
        urldecode = parse.unquote(params_decode)
        
        # 사용자 요청 데이터
        response = urldecode.split('=')[1]
        response_list = []
        response_list.append(response)
        
        word_ = tfidf_Vectorizer(response_list)
        
        similar_list = similar_keyword(word_)
        MySQL_data_save(response)
        top10_list = keyword_rank(word_)
        
        SNU_title, SNU_score = SNU_Search(word_)
        Naver_title, Naver_link, Naver_score = Naver_Search(response)
        

        
        data_ = {
            'response':response,
            'Naver_title':Naver_title
            }
        
        print('입력 데이터 : \n', response)
        print('\n')
        print('연관 검색어 : \n', similar_list)
        print('\n')
        print('네이버 뉴스 제목 : \n', Naver_title)
        print('\n')
        print('네이버 뉴스 링크 : \n', Naver_link)
        print('\n')
        print('네이버 뉴스 score : \n', Naver_score)
        print('\n')
        try:
            print('SNU 기사 제목 : \n', SNU_title)
            print('\n')
    #         print('SNU 기사 링크 : ', link_list2)
            print('SNU 기사 score : \n', SNU_score)
            print('\n')
        except:
            print('SNU 기사가 없습니다. \n')
        print('인기키워드 랭킹 : \n', top10_list)
        print('\n')
        print('\n')
        
        return json.dumps(data_, ensure_ascii=False)

if __name__ == '__main__':
    app.run(host='0.0.0.0', port = 8080)

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://0.0.0.0:8080/ (Press CTRL+C to quit)


(1, 1)
{'vdsadzdzcsda': 0}
             단어   빈도
0  vdsadzdzcsda  1.0 



125.133.174.241 - - [09/Sep/2021 08:57:18] "POST / HTTP/1.1" 200 -


입력 데이터 : 
 vdsadzdzcsda


연관 검색어 : 
 ['재밋다', '밋다', '웃기', '후지', '잼', '재밋음', '재밌다', '재미없다', '재밋습', 'ㅛ']


네이버 뉴스 제목 : 
 []


네이버 뉴스 링크 : 
 []


네이버 뉴스 score : 
 []


SNU 기사 제목 : 
 []


SNU 기사 score : 
 []


인기키워드 랭킹 : 
 ['안녕하세요', '유료', '카카오', '25', '코로나', '123123', '카카오톡', 'nameasdfasdfas', '카카오톡 유료화', 'afsdfafdads']




